In [10]:
import pandas as pd
import numpy as np

Загружаю датасет с продажами серебра и золота отдельно. Цель - определить маги для продаж золота, по прогнозу от модели на продажи серебра. Соответственно нужно выявить есть ли корреляция между продажами золота и серебра.


In [90]:
mus_sales = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='олап', skiprows=10, usecols=[0,2,3], names=['id','585_msale','925_msale'])

# убираем пустые значения
lst = mus_sales['id'].tolist()
mus_sales = mus_sales.drop(axis=0,index=np.arange(lst.index('Общий итог'),len(lst)))
mus_sales.tail()

,id,585_msale,925_msale
852,91,1.0,1.0
853,925,0.0,2.0
854,955,0.0,6.0
855,96,3.0,10.0
856,99,6.0,8.0


In [107]:
mus_stock = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='олап', skiprows=10, usecols=[7,9,10],names=['id','585_mstock','925_mstock'])


# убираем пустые значения
lst = mus_stock['id'].tolist()
mus_stock = mus_stock.drop(axis=0,index=np.arange(lst.index('Общий итог'),len(lst)))

# убираем магазины с нулевыми остатками
mus_stock['sum'] = mus_stock['585_mstock'] + mus_stock['925_mstock']
mus_stock = mus_stock[(mus_stock['sum'] > 0)]
del mus_stock['sum']
mus_stock.tail()

,id,585_mstock,925_mstock
1426,92,16.0,8.0
1427,925,1.0,8.0
1428,955,6.0,12.0
1429,96,15.0,7.0
1432,99,25.0,15.0


In [93]:
#оставляем склады, где есть остатки золото и серебро одновременно

mus_stock_combined = mus_stock[(mus_stock['585_mstock'] > 5)&(mus_stock['925_mstock'] > 5)].drop(axis=1, labels=['585_mstock','925_mstock'])

#посмотрим корреляцию в продажах золота и серебра
mus_sales_combined = pd.merge(mus_stock_combined, mus_sales, on='id',how='left').dropna(how='any')
mus_sales_combined[['585_msale','925_msale']].corr()

,585_msale,925_msale
585_msale,1.000000,0.683501
925_msale,0.683501,1.000000


In [89]:
mus_sales_combined.describe()

,585_sal,925_sal
count,468.000000,468.000000
mean,6.814103,8.087607
std,6.600050,7.846685
min,0.000000,0.000000
25%,2.000000,3.000000
50%,4.500000,6.000000
75%,10.000000,10.000000
max,39.000000,60.000000


Коэффицент корреляции между высоким и заметным. Попробуем пользоваться моделью для предсказания продаж мусульманских подвесов из серебра, т.к. они присутствуют во всех магазинах, а золотые подвесы распределяются только в магазины с хорошими продажами по серебру.

Создаем большой датасет для обучения

In [123]:
#загружаем продажи обычного ассортимента
sales = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='олап', skiprows=10, usecols=[12,14,15], names=['id','585_sale','925_sale'])

# убираем пустые значения
lst = sales['id'].tolist()
sales = sales.drop(axis=0,index=np.arange(lst.index('Общий итог'),len(lst)))
sales.describe()

,585_sale,925_sale
count,913.000000,913.000000
mean,516.859803,879.210296
std,272.360040,447.028022
min,-4.000000,-22.000000
25%,340.000000,593.000000
50%,488.000000,824.000000
75%,664.000000,1078.000000
max,2523.000000,3467.000000


In [120]:
#загружаем остатки обычного ассортимента
stock = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='олап', skiprows=10, usecols=[19,21,22], names=['id','585_stock','925_stock'])

# убираем пустые значения
lst = stock['id'].tolist()
stock = stock.drop(axis=0,index=np.arange(lst.index('Общий итог'),len(lst)))

# убираем магазины с нулевыми остатками
stock['sum'] = stock['585_stock'] + stock['925_stock']
stock = stock[(stock['sum'] > 0)]
del stock['sum']
stock.describe()

,585_stock,925_stock
count,955.000000,955.000000
mean,1990.301571,2434.910995
std,938.306706,1115.372661
min,0.000000,0.000000
25%,1470.500000,1716.500000
50%,1972.000000,2337.000000
75%,2498.500000,3168.000000
max,5620.000000,8325.000000


In [137]:
# соединяем продажи остатки мусульманский ассортимент и обычный
df_all = stock.merge(sales.merge(mus_stock.merge(mus_sales,on='id',how='left'),on='id',how='left'),on='id',how='left')
df_all.describe()

,585_stock,925_stock,585_sale,925_sale,585_mstock,925_mstock,585_msale,925_msale
count,955.000000,955.000000,910.000000,910.000000,893.000000,893.000000,844.000000,844.000000
mean,1990.301571,2434.910995,518.338462,881.624176,14.793953,13.723404,4.109005,6.215640
std,938.306706,1115.372661,271.531989,445.686217,16.591440,9.274022,5.959329,6.751719
min,0.000000,0.000000,-4.000000,-22.000000,0.000000,0.000000,0.000000,0.000000
25%,1470.500000,1716.500000,341.000000,595.250000,0.000000,8.000000,0.000000,2.000000
50%,1972.000000,2337.000000,488.500000,824.500000,14.000000,11.000000,2.000000,4.000000
75%,2498.500000,3168.000000,664.750000,1078.000000,26.000000,17.000000,6.000000,8.000000
max,5620.000000,8325.000000,2523.000000,3467.000000,104.000000,61.000000,39.000000,60.000000


In [138]:
#загружаем дополнительную информацию по объектам и соединяем ее с основным датасетом
stores = pd.read_excel(
    r'C:\нетология\muslim.xlsx', sheet_name='типы', usecols=[0,1,5,7,8])
stores['склад'] = stores['склад'].astype(str)
df_all = pd.merge(df_all,stores,left_on='id',right_on='склад',how='left')
df_all.describe()

,585_stock,925_stock,585_sale,925_sale,585_mstock,925_mstock,585_msale,925_msale,Год
count,957.000000,957.000000,912.000000,912.000000,895.000000,895.000000,846.000000,846.000000,899.000000
mean,1987.877743,2432.117032,518.038377,881.782895,14.788827,13.706145,4.108747,6.216312,2014.276974
std,938.867189,1115.884998,271.347044,445.234758,16.583768,9.270947,5.955455,6.749045,3.302498
min,0.000000,0.000000,-4.000000,-22.000000,0.000000,0.000000,0.000000,0.000000,2006.000000
25%,1466.000000,1715.000000,340.750000,595.750000,0.000000,8.000000,0.000000,2.000000,2013.000000
50%,1971.000000,2336.000000,488.000000,825.500000,14.000000,11.000000,2.000000,4.000000,2014.000000
75%,2496.000000,3166.000000,664.250000,1078.000000,26.000000,17.000000,6.000000,8.000000,2017.000000
max,5620.000000,8325.000000,2523.000000,3467.000000,104.000000,61.000000,39.000000,60.000000,2019.000000


очищаем от магазинов с маленькими нормами, с отсутствием продаж по золоту

Разбиваем магазины на группы по размеру

Добавить среднюю зарплату, тип города по количеству населения, процент мусульман, количество трудовых мигрантов, культурных объектов мусульманской тематики по городам 

достать из контакта мусульман по городам

In [ ]:
добавить широту  долготу город область и т.д.

In [31]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Yandex") 
location = geolocator.geocode("Рузаевка")
print(location)

Рузаевка, Рузаевский район, Мордовия, Приволжский федеральный округ, РФ


In [27]:
(geolocator.geocode("Владикавказ, РФ").latitude, geolocator.geocode("Владикавказ, РФ").longitude)

(43.024593, 44.68211)

In [146]:
lst=stores['Город'].unique().tolist()
lst[0:3]

['Кемерово', 'Киселевск', 'Междуреченск']

In [149]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Google")
location = geolocator.geocode(lst[0:3])
location.address

AttributeError: 'NoneType' object has no attribute 'address'